In [1]:
import os
os.chdir("..")

from torchvision.transforms import Lambda, Compose

from wf_vae.model import *
from wf_vae.dataset.transform import ToTensorVideo, CenterCropResizeVideo

from model.flameObj import *
from model.causalvideovae.model import *

import trimesh

os.getcwd()

'/nfs/horai.dgpsrv/ondemand28/harryscz/diffusion'

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
flamePath = "/scratch/ondemand28/harryscz/head_audio/head/code/flame/flame2023_no_jaw.npz"
sourcePath = "/scratch/ondemand28/harryscz/head_audio/head/data/vfhq-fit"
dataPath = [os.path.join(os.path.join(sourcePath, data), "fit.npz") for data in os.listdir(sourcePath)]
seqPath = "/scratch/ondemand28/harryscz/head/_-91nXXjrVo_00/fit.npz"

In [ ]:
%load_ext autoreload
%autoreload 2
res=256
head = Flame(flamePath, device="cuda")
head.loadSequence(dataPath[0])
seq = head.LSB(rotation=False)
head.get_3d_animation(seq, "test/orig3d.mp4", dist=1.2, resolution=res)
uvMesh = head.convertUV(rotation=False)
uv = head.get_uv_animation(uvMesh, savePath="test/origUV.mp4", sample_frames=800,  resolution=res, fill=True)
uv = uv[..., :3].permute(3,0,1,2)[:,:100,...].unsqueeze(0)
uv.shape

/nfs/horai.dgpsrv/ondemand28/harryscz/pytorch3d/pytorch3d/io/obj_io.py:551: UserWarning: Mtl file does not exist: /scratch/ondemand28/harryscz/head_audio/head/template.mtl
  warnings.warn(f"Mtl file does not exist: {f}")


torch.Size([1, 3, 100, 512, 512])

In [4]:
model_cls = ModelRegistry.get_model("WFVAE")
vae = model_cls.from_pretrained("/scratch/ondemand28/harryscz/other/WF-VAE/weight")

In [5]:
ckpt_path = "/scratch/ondemand28/harryscz/diffusion/modelOut/vae/WFVAE-lr1.00e-05-bs1-rs256-sr1-fr25/checkpoint-44000.ckpt"
ckpt = torch.load(ckpt_path, map_location="cpu")
vae.load_state_dict(ckpt["state_dict"], strict=False)
vae = vae.to(device="cuda", dtype=torch.float32)

/tmp/ipykernel_696300/932873551.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_path, map_location="cpu")


In [6]:
with torch.no_grad():
    z = vae.encode(uv.to("cuda")) # input shape : C, T, H, W
    latents = z.latent_dist.sample()
    recon = vae.decode(latents).sample
    
uv.shape, recon.shape, latents.shape

(torch.Size([1, 3, 100, 512, 512]),
 torch.Size([1, 3, 97, 512, 512]),
 torch.Size([1, 16, 25, 64, 64]))

In [7]:
sampleRecon = head.sampleFromUV(uv[0].permute(1,2,3,0), savePath="test/sampleRecon.mp4", resolution=512, fill=False)
# sampleRecon = moving_average_smoothing(sampleRecon, kernel_size=2)

x = head.sampleTo3D(sampleRecon, savePath="test/3dRecon.mp4", dist=1.2, azim=0, elev=0, resolution=512)

In [6]:
def moving_average_smoothing(motion, kernel_size=5):
    # motion: [T, V, D]
    motion = motion.permute(1, 2, 0)  # [V, D, T]
    motion = F.pad(motion, (kernel_size // 2, kernel_size // 2), mode='reflect')  # Temporal padding
    motion = motion.unfold(-1, kernel_size, 1).mean(dim=-1)  # Apply moving average
    motion = motion.permute(2, 0, 1)  # [T, V, D]
    return motion

def build_neighbor_index(faces: torch.LongTensor, V: int, k: int):
    """
    faces: [F,3]     triangle list
    V:      int       number of vertices
    k:      int       max nbrs per vertex
    returns: LongTensor [V,k] neighbor indices (padded with self)
    """
    nbrs = [[] for _ in range(V)]
    for i,j,l in faces.tolist():
        nbrs[i].extend([j,l])
        nbrs[j].extend([i,l])
        nbrs[l].extend([i,j])
    idx = torch.empty((V, k), dtype=torch.long)
    for v, nl in enumerate(nbrs):
        if len(nl) >= k:
            chosen = nl[:k]
        else:
            chosen = nl + [v] * (k - len(nl))
        idx[v] = torch.tensor(chosen, dtype=torch.long)
    return idx  # [V,k]

def fast_fix_spikes(verts: torch.Tensor,
                    neighbor_idx: torch.LongTensor,
                    threshold: float):
    """
    verts:        FloatTensor [T, V, 3]
    neighbor_idx: LongTensor  [V, k]
    threshold:    float
    """
    # ensure indices live on the same device
    neighbor_idx = neighbor_idx.to(verts.device)  # [V,k]
    
    # 1) Gather neighbor coords by advanced indexing:
    #    verts[:, neighbor_idx, :] -> shape [T, V, k, 3]
    nbr_feats = verts[:, neighbor_idx, :]          

    # 2) Compute per-vertex mean over k neighbors: [T, V, 3]
    nbr_mean = nbr_feats.mean(dim=2)               

    # 3) Spike‑detect: distance > threshold → [T, V]
    dists = (verts - nbr_mean).norm(dim=2)         
    mask  = dists > threshold                      

    # 4) Replace only spikes:
    fixed = torch.where(
        mask.unsqueeze(-1),   # [T, V, 1]
        nbr_mean,             # if spike → replace
        verts                 # else → keep
    )
    return fixed


In [9]:
# head.sampleFromUV(uv[0].permute(1,2,3,0), savePath="test/sampleOrig.mp4")
%load_ext autoreload
%autoreload 2
# head = Flame(flamePath, device="cuda")
# sampleRecon = head.sampleFromUV(recon[0].permute(1,2,3,0), savePath="test/sampleRecon.mp4", resolution=512)
sampleRecon = head.sampleFromUV(uv[0].permute(1,2,3,0), savePath="test/sampleRecon.mp4", resolution=512, fill=False)
# sampleRecon = moving_average_smoothing(sampleRecon, kernel_size=2)

x = head.sampleTo3D(sampleRecon, savePath="test/3dRecon.mp4", dist=1.2, azim=30, elev=40, resolution=512)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
gMin = torch.tensor([-0.1124, -0.1944, -0.1546], dtype=torch.float32).to(head.device)
gMax = torch.tensor([0.1093, 0.1337, 0.0895], dtype=torch.float32).to(head.device)

seq = (seq - gMin) / (gMax - gMin)

gMin = torch.tensor([-0.1776, -0.2074, -1.0983], dtype=torch.float32).to(head.device)
gMax = torch.tensor([ 0.1772,  0.1642, -0.6894], dtype=torch.float32).to(head.device)

seq = (seq * (gMax - gMin) + gMin) * 3

In [44]:
neighbor_idx = build_neighbor_index(head.faces3d.cpu(), V=head.faces3d.max().item()+1, k=9)

fixed = fast_fix_spikes(x, neighbor_idx, threshold=0.11)
v = head.render3dAnimation(fixed, "test/test.mp4", resolution=512, dist=1.2, azim=35, elev=20)

faces = head.faces3d.cpu().numpy()
verts = fixed[35, ...].cpu().numpy()

trimesh.Trimesh(vertices=verts, faces=faces, process=True).export("test/__i.ply")

b'ply\nformat binary_little_endian 1.0\ncomment https://github.com/mikedh/trimesh\nelement vertex 4525\nproperty float x\nproperty float y\nproperty float z\nelement face 9976\nproperty list uchar int vertex_indices\nend_header\n\xaa\xb0\xad\xbd\xf7)\x9b\xbe\xd0\x07A\xc0\xc0"\xbc\xbdT\x10\x99\xbe\xcc\'B\xc0\xc2\x97\xb4\xbd\x9a\xc8\x96\xbe\xc3\xa1A\xc0\x06J\xaf\xbd\x8a\xa6\x95\xbe\xe0\xb8A\xc0 @\xae\xbd\xab\xb3\x95\xbe\xd6\xe0A\xc0\xb6\x189\xbe2(\xbb\xbe\x8e\xdbJ\xc0\xd1YZ\xbe\x0c\xe9\xba\xbe\xd3>L\xc0HW]\xbe\xf6\xba\xc4\xbe\xa2\xaeK\xc0\x17\xb6?\xbeLd\xc6\xbe\x1f^J\xc0A\x1a\x1b\xbe\xc9[\xda\xbe\xb2\rG\xc0\xa0\xb7$\xbeJ\xc9\xd3\xbe\xe6\xefG\xc0<\xc9*\xben8\xde\xbe\xaf\xa5H\xc0\xc1\x1c\x17\xbe4d\xe1\xbeL\xdfF\xc0r\x824\xber\xb6P\xbe\x8cr3\xc0|\x80#\xbe:\xfaS\xbeX:4\xc0\xac\xb3$\xbe\x99\x07]\xbe\x92\xd13\xc0\x18\x89\xdf\xbe\x90\xf0\x9a\xbe\xa1\xb4A\xc0|\xe6\xde\xbe\x89|\x99\xbe\x15\xe8B\xc0{\xa0\xe0\xbe \xf5\x96\xbe~CB\xc0J\xe2\xe1\xbe\x85\xa9\x95\xbe\xbe9B\xc0\xfa(\xe0\xbe\x8e\x82\x95\xb

In [31]:
verts = x[35, ...].cpu().numpy()
faces = head.faces3d.cpu().numpy()

trimesh.Trimesh(vertices=verts, faces=faces, process=True).export("test/__i.ply")

b'ply\nformat binary_little_endian 1.0\ncomment https://github.com/mikedh/trimesh\nelement vertex 4468\nproperty float x\nproperty float y\nproperty float z\nelement face 9976\nproperty list uchar int vertex_indices\nend_header\n.\xfe\xb7\xbd\x00\xe3\x9a\xbe)fA\xc0\xd6;\xb1\xbdq<\x9b\xbeFLA\xc0>\xbf\xb9\xbd\x99\x82\x99\xbe\x80\x93B\xc0\x1eg\xb3\xbd\xf7\xfa\x96\xbe\n\xedA\xc0 @\xae\xbd\xab\xb3\x95\xbe\xd6\xe0A\xc0\x80-\xb5\xbd\x1e\x8b\x95\xbeL\'B\xc0\xc0\xf8?\xbe\x8a(\xbb\xbez;K\xc0\xcd?c\xbe\xcc\x11\xbb\xbe\x91hL\xc0\xdcZb\xbeh\xe7\xc4\xbe7\xccK\xc0f\xb4C\xbeB\x86\xc6\xbe\xee\x92J\xc0%M\x1c\xbe\x12g\xda\xbe./G\xc0\x1ad\'\xbe\xf4\xe6\xd3\xbe\x070H\xc0\x83)-\xbe\xcbj\xde\xbe\xd5\xdcH\xc0\xe0@\x19\xbe>m\xe1\xbe\xf8\x1fG\xc0\x96`3\xbe\xfapP\xbeP\x803\xc0\xf8z"\xbe\xf9\xaaS\xbe\x9eL4\xc0\xac\xb3$\xbe\x99\x07]\xbe\x92\xd13\xc0\xb2@\xdc\xbe\x0c\xdc\x99\xbev\xd7A\xc0h\xef\xdd\xbe{z\x9a\xber\xc8A\xc0\xa0\xd1\xde\xbe\x12{\x99\xbePYC\xc0\xe0\xd4\xdf\xbeP\x01\x97\xbe\x81\x8eB\xc0\xfa(\xe0\xbe\x8e\